In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics
N = 100
X = np.sort(20 * np.random.rand(N, 1)-10, axis=0)
y = np.sin(X)/X
y=y.ravel()
noise = np.random.normal(0,0.2,N)
t = y + noise
#print(t)
#y[::5] += 0.2 * (0.5 - np.random.rand(8)) #a = np.zeros((3, 3), int)
A = np.zeros((N+1, N+1))

np.fill_diagonal(A, 1e-6)  # initialoze all alpha is 0.01
B = np.zeros((N, N))
np.fill_diagonal(B, 1e-6)
Alpha = np.diag(A)
PHI =metrics.pairwise.rbf_kernel(X.reshape(-1,1),X.reshape(-1,1))
#PHI=np.r_[np.ones((N,1)),PHI]
PHI=np.column_stack((np.ones(N),PHI)) # N by N+1


for i in range(1,1000):
    #U = np.linalg.cholesky(1/0.04 * np.dot(PHI.T, PHI) + A)
    #Ui = np.linalg.inv(U)
    
    #Sigma = np.dot(Ui,Ui.T)
    #Sigma = np.linalg.inv((1/0.04 * np.dot(PHI.T, PHI) + A))
    #Mu = 1/0.04 * np.dot(np.dot(Sigma, PHI.T),t)
    Sigma = np.linalg.inv(np.dot(np.dot(PHI.T, B),PHI) + A) # N+1 by N+1
    Mu = np.dot(np.dot(np.dot(Sigma,PHI.T),B),t.reshape(-1,1))  # N+1 by 1
    
    #print(np.diag(Sigma).reshape(1,-1))
    
    #Gamma = 1 -  np.dot(np.diag(A).reshape(-1,1), np.diag(Sigma).reshape(1,-1))
    #A = np.diag(Gamma)/np.square(Mu)
    Gamma = 1-np.diag(A).reshape(-1,1)* np.diag(Sigma).reshape(-1,1)
    #Gamma = 1- Alpha * np.sum(np.square(Ui),axis=0)
    #print("Gamma is : ", Gamma)
    #DIF = Alpha - Gamma/np.square(Mu)/Gamma/np.square(Mu)
    #print(DIF)
    Alpha = Gamma/np.square(Mu)
    #print(len(Alpha))
    np.fill_diagonal(A,Alpha)
    #print("A is: ", A)    
    #Sigma = np.linalg.inv(np.dot(np.dot(PHI.T, B),PHI) + A) # N+1 by N+1
    #Mu = np.dot(np.dot(np.dot(Sigma,PHI.T),B),t.reshape(-1,1))  # N+1 by 1
    #Sigma_new = np.sum(np.square(t - np.dot(PHI,Mu)))/(N-np.sum(Gamma))
    #Sigma_new = np.sum(np.square(t - np.dot(PHI,Mu)))/(N-np.sum(Gamma))
    
    Sigma_new = np.dot(np.absolute(t-np.dot(PHI,Mu).T),np.absolute(t-np.dot(PHI,Mu).T).T)/(N-np.sum(Gamma))
    #print(Sigma_new)
    np.fill_diagonal(B,1/Sigma_new)
    #print(B)
    #print('Sigma is :',np.sqrt(Sigma_new))
#print('alpha is :',A)
#print(np.dot(PHI,Mu).T)
print('Sigma is :',np.sqrt(Sigma_new))
Mu[Mu<1e-5]=0
#print(Mu)
#print(np.nonzero(Mu))

In [ ]:
W = Mu[np.nonzero(Mu)]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics
from sklearn.utils.validation import check_X_y



# generate data
N = 10
X = np.sort(20 * np.random.rand(N, 1)-10, axis=0)
y = np.sin(X)/X
y=y.ravel()

#noise = np.random.normal(0,0.2,N) # Beta = sigma=0.2 sigma ^-2 = 25
#noise=[-0.20499857, -0.60856184,  0.01401506, -0.02762508,  0.04610335,  0.27162553,
# -0.13175594,  0.03615839, -0.10797591,  0.26634466]

noise = [-0.07849931,  0.42809446, -0.10594689, -0.00051753, -0.12775751,  0.05353166,
 -0.1379147,  -0.19749898, -0.0194281,   0.50225254]
t = y + noise
y = y.reshape(-1,1)
X = X.reshape(-1,1) # N by 1
t = t.reshape(-1,1) # N by 1
X, y = check_X_y(X, t)
# 
print(X)
print(y)
Phi =metrics.pairwise.rbf_kernel(X, X) # Gaussian kernel 
#Phi =metrics.pairwise.linear_kernel(X, X)
#PHI=np.column_stack((np.ones(N),Phi)) 

#PHI=np.column_stack((Phi, np.ones(N))) 

PHI = np.append(Phi, np.ones((Phi.shape[0], 1)), axis=1)
n_basic_functions = PHI.shape[1]
print(n_basic_functions)
# initial 
Alpha = 1e-6 * np.ones(n_basic_functions) 
Beta =  1.e-6# sigma^-2 = 1e-6 
Threshod_Alpha = 1e10
Threshold_Delta = 1e-3
# fit 
Alpha_old = Alpha
Mu=np.zeros(n_basic_functions)

for i in range(10):
    # posterior():
    i_s = Beta * np.dot(PHI.T, PHI) + np.diag(Alpha)
    Sigma = np.linalg.inv(i_s)
    #Mu = Beta * np.dot(np.dot(Sigma, PHI.T), t)
    Mu = Beta * np.dot(Sigma, np.dot(PHI.T, t))
    Gamma = 1 -  Alpha * np.diag(Sigma)  
    Alpha = Gamma / (Mu ** 2)
    #print(np.sum(Gamma))
    #print(Gamma)
    print(Mu)
    Beta = (N - np.sum(Gamma))/(np.sum((t - np.dot(PHI, Mu))**2))
    #print("Gamm is ", Gamma)
    #print("Alpha is ", Alpha)
    #print("Beta is ", Beta)
    # prune():
    Keep_Alpha = Alpha < Threshod_Alpha
    #if len(Keep_Alpha)<10:
        #break
     
    if not np.any(Keep_Alpha):
    
        Keep_Alpha[0] = True    
        Keep_Alpha[-1] = True
    #if not Keep_Alpha[-1]:
        
    Alpha = Alpha[Keep_Alpha]
    print("Gamma: {}".format(Gamma))
    
    Alpha_old = Alpha_old[Keep_Alpha]
    Gamma = Gamma [Keep_Alpha]

    PHI = PHI[:,Keep_Alpha]
    Sigma = Sigma[np.ix_(Keep_Alpha,Keep_Alpha)]
    Mu = Mu[Keep_Alpha]
    #print(i)
    print("i_s: {}".format(i_s))
    print("sigma: {}".format(Sigma))
    print("mu {}".format(Mu))
    print("Iteration: {}".format(i))
    print("Alpha: {}".format(Alpha))
    print("Beta: {}".format(Beta))
    print("Gamma: {}".format(Gamma))
    print("m: {}".format(Mu))
    #print("Relevance Vectors: {}".format(self.relevance_.shape[0]))
    print()
    
    
    Delta = np.amax(np.absolute(Alpha-Alpha_old))
    if Delta < Threshold_Delta and i > 1:
        break
        
    Alpha_old = Alpha    
#Beta = (N - np.sum(Gamma))/np.sum((t - np.dot(PHI, Mu))**2)
print(i)    
print("Beta is ", Beta)
print("Alpha is ", Alpha)
print("Mu is ", Mu)    
                                    
                                    

In [70]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils.validation import check_X_y
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import BaseEstimator, RegressorMixin, ClassifierMixin
class BaseRVM(BaseEstimator):
    
    def __init__(
        self,
        kernel='rbf',
        #degree=3,
        coef1=None,
        #coef0=0.0,
        n_iter=3000,
        tol = 1e-3,
        alpha=1e-6,
        threshold_alpha=1e9,
        beta=1.e-6,
        beta_fixed=False,
        bias_used=True,
        verbose=False
):
        self.kernel = kernel
        #self.degree = degree
        self.coef1 = coef1
        #self.coef0 = coef0
        self.n_iter = n_iter
        self.tol = tol
        self.alpha = alpha
        self.threshold_alpha = threshold_alpha
        self.beta = beta
        self.beta_fixed = beta_fixed
        self.bias_used = bias_used
        self.verbose = verbose
        
    def get_params(self, deep=True):
        params={
            'kernel': self.kernel,
            'degree': self.degree,
            'coef1': self.coef1,
            #'coef0': self.coef0,
            'n_iter': self.n_iter,
            'tol': self.tol,
            'alpha': self.alpha,
            'threshold_alpha': self.threshold_alpha,
            'beta': self.beta,
            'beta_fixed': self.beta_fixed,
            'bias_used': self.bias_used,
            'verbose': self.verbose
        }
        return params
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
        
    def apply_kernel(self, x, y):
        if self.kernel == 'rbf':
            phi = rbf_kernel(x, y, self.coef1) # Gaussian kernel 
        if self.bias_used:
            phi = np.append(phi, np.ones((phi.shape[0], 1)), axis=1)   
        return phi
    
    def _prune(self):
        keep_alpha = self.alpha_ < self.threshold_alpha
        
        
        #if not np.any(keep_alpha):
         #   print("!!!")
         #   keep_alpha[0]=True
         #   if self.bias_used:
        
                
        #if self.bias_used:
        #    if not keep_alpha[-1]:
        #        self.bias_used=False
        #    self.relevance_ = self.relevance_[keep_alpha[:-1]]
        #else:
        #    self.relevance_ = self.relevance_[keep_alpha]
        
        
        self.alpha_ = self.alpha_[keep_alpha]
        self.alpha_old = self.alpha_old[keep_alpha]
        self.gamma = self.gamma[keep_alpha]
        print("phi pruned before: {}".format(self.phi))
        self.phi = self.phi[:, keep_alpha]
        print("phi pruned after: {}".format(self.phi))
        
        print("sigma pruned before: {}".format(self.sigma_))
        self.sigma_ = self.sigma_[np.ix_(keep_alpha, keep_alpha)]
        print("sigma pruned after: {}".format(self.sigma_))
        self.m_ = self.m_[keep_alpha]

        
    def fit(self, X, y):
        X, y = check_X_y(X, y)
        n_samples, n_features = X.shape
        self.phi = self.apply_kernel(X, X)
        n_basis_functions = self.phi.shape[1] 
        self.relevance_ = X
        self.y=y
        
        self.alpha_ = self.alpha * np.ones(n_basis_functions)
        self.beta_ = self.beta
        
        self.m_ = np.zeros(n_basis_functions)
        
        self.alpha_old = self.alpha_
        
        for i in range(self.n_iter):
            self.posterior()
            
            self.gamma = 1 - self.alpha_ * np.diag(self.sigma_)
            self.alpha_ = self.gamma/(self.m_ ** 2)
            
            if not self.beta_fixed:
                self.beta_ = (n_samples - np.sum(self.gamma))/(np.sum((y-np.dot(self.phi, self.m_))**2))
            
            self._prune()
        
            if self.verbose:
                print("Iteration: {}".format(i))
                print("Alpha: {}".format(self.alpha_))
                print("Beta: {}".format(self.beta_))
                print("Gamma: {}".format(self.gamma))
                print("m: {}".format(self.m_))
                print("Relevance Vectors: {}".format(self.relevance_.shape[0]))
                print()
            
            delta = np.amax(np.absolute(self.alpha_ - self.alpha_old))
        
            if delta < self.tol and i > 1:
                break
            
            self.alpha_old = self.alpha_
        
        if self.bias_used:
            self.bias = self.m_[-1]
        else:
            self.bias = None
        return self

class RVR(BaseRVM, RegressorMixin):
        
    def posterior(self):
        i_s = np.diag(self.alpha_) + self.beta_ * np.dot(self.phi.T, self.phi)
        self.sigma_ = np.linalg.inv(i_s)
        #self.m_ = self.beta_ * np.dot(self.sigma_, np.dot(self.phi.T, self.y))
        self.m_ = self.beta_ * np.dot(np.dot(self.sigma_,self.phi.T),self.y)
    def predict(self, X, y, eval_MSE=False):
        phi = self.apply_kernel(X, self.relevance_)
        y = np.dot(phi, self.m_)
        if eval_MSE:
            MSE = (1/self.beta_) + np.dot(phi, np.dot(self.sigma_, phi.T))
            return y, MSE[:,0]
        else:
            return y
        

In [72]:

def generate_data(N, sigma):
    X = np.sort(20 * np.random.rand(N, 1)-10, axis=0)
    y = np.sin(X)/X
    y=y.ravel()
    noise = np.random.normal(0, sigma, N)
    t = y + noise
    return X, t

#print(y)
rbf_rvm = RVR(kernel='rbf', verbose=False, n_iter=5, bias_used=True)
#rbf_rvm = RVR(kernel='rbf')
X, y = generate_data(10, 0.2)
#rbf_rvm.fit(X, y)
#print(rbf_rvm.beta_)
y_rbf = rbf_rvm.fit(X,y)

phi pruned before: [[  1.00000000e+00   2.58832886e-01   3.39557546e-10   1.46359821e-27
    1.32134733e-38   1.57301952e-54   4.11977533e-55   9.37844634e-65
    1.01050608e-71   1.80762317e-77   1.00000000e+00]
 [  2.58832886e-01   1.00000000e+00   4.56128254e-06   3.28126870e-20
    9.21876895e-30   7.06695406e-44   2.12805803e-44   4.42755475e-53
    2.13047745e-59   1.24016040e-64   1.00000000e+00]
 [  3.39557546e-10   4.56128254e-06   1.00000000e+00   3.77831260e-05
    3.38528384e-10   7.42050777e-19   3.40421611e-19   5.60754226e-25
    2.46314710e-29   5.03750851e-33   1.00000000e+00]
 [  1.46359821e-27   3.28126870e-20   3.77831260e-05   1.00000000e+00
    1.12352645e-01   2.27287372e-05   1.52944607e-05   1.09385717e-08
    2.92162855e-11   1.52385132e-13   1.00000000e+00]
 [  1.32134733e-38   9.21876895e-30   3.38528384e-10   1.12352645e-01
    1.00000000e+00   4.04078428e-02   3.24720860e-02   3.87286484e-04
    6.93811658e-06   1.62280664e-07   1.00000000e+00]
 [  1.57301

In [63]:
1/np.sqrt(y_rbf.beta_)

0.20745031181500198

In [49]:
a=[1, 2, 3, 4, 5]
print(a[:-1])

[1, 2, 3, 4]


In [51]:
#print(y_rbf.relevance_)
print(y_rbf.alpha_)

[  5.04534907e+00   6.97138230e+00   1.73853565e+00   1.66285592e+00
   2.85852164e+01   4.25334951e+04   7.25844986e+00   4.90580338e+01
   1.31336716e+02   2.10481391e+02]


In [64]:
x= [1,2,3,4]
print(np.diag(x))

[[1 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 4]]


In [65]:
a = np.diag(x)
print(np.diag(a))

[1 2 3 4]


In [67]:
x[1:]

[2, 3, 4]

In [84]:
keep = [True, True, True, True, True]
x=[1,2,3,4]
a=np.diag(x)
y=[1,2,3,4,5]
b=np.diag(y)
a = np.append(a,np.ones((4,1)),axis=1)
print(a)
print(b[:,keep])
#print(a[:,keep])
#print(a[np.ix_(keep,keep)])
print(b[np.ix_(keep,keep)])

[[ 1.  0.  0.  0.  1.]
 [ 0.  2.  0.  0.  1.]
 [ 0.  0.  3.  0.  1.]
 [ 0.  0.  0.  4.  1.]]
[[1 0 0 0 0]
 [0 2 0 0 0]
 [0 0 3 0 0]
 [0 0 0 4 0]
 [0 0 0 0 5]]
[[1 0 0 0 0]
 [0 2 0 0 0]
 [0 0 3 0 0]
 [0 0 0 4 0]
 [0 0 0 0 5]]
